In [1]:
# import os
# os.getcwd()

In [60]:
!pip3 install konlpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 19.4 MB 16.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 33.1 MB/s eta 0:00:01
     |████████████████████████████████| 449 kB 33.6 MB/s eta 0:00:01


In [64]:
!apt install -y default-jdk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  ca-certificates-java default-jdk-headless default-jre default-jre-headless
  fonts-dejavu-extra java-common libasound2 libasound2-data
  libatk-wrapper-java libatk-wrapper-java-jni libfontenc1 libice-dev
  libpcsclite1 libpthread-stubs0-dev libsm-dev libx11-dev libx11-doc
  libxau-dev libxaw7 libxcb-shape0 libxcb1-dev libxdmcp-dev libxmu6 libxt-dev
  libxv1 libxxf86dga1 openjdk-11-jdk openjdk-11-jdk-headless openjdk-11-jre
  openjdk-11-jre-headless x11-utils x11proto-core-dev x11proto-dev
  xorg-sgml-doctools xtrans-dev

In [77]:
!pip3 install gensim

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 24.1 MB 7.7 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 2.6 MB/s  eta 0:00:01


In [1]:

# !git clone https://github.com/kiyoungkim1/ReadyToUseAI

from ReadyToUseAI.src.nlp import make_sample_dataset, bert_sequence_classification
# make_sample_dataset.nsmc(mode='test', text_only=False)  # mode: which datasets? 'train' or 'test'

In [72]:
import pandas as pd
import numpy as np
train = pd.read_csv('/USER/3_WEEK/MNC_NLP/train/train.csv')
combinations = np.array(np.meshgrid(train.bias.unique(), train.hate.unique())).T.reshape(-1,2)
bias_hate = list(np.array([train['bias'].values, train['hate'].values]).T.reshape(-1,2))
labels = []
for i, arr in enumerate(bias_hate):
    for idx, elem in enumerate(combinations):
        if np.array_equal(elem, arr):
            labels.append(idx)

train['label'] = labels
# train = train.drop(columns=["title", "bias", "hate"])
# train = train.set_axis(['content','label'], axis=1, inplace=False)

import re
def remove_special_char(text):
    text = re.sub('[^ㄱ-ㅣ가-힣 0-9]+',' ', str(text))
    return text

train.title = train.title.apply(remove_special_char)
train.comment = train.comment.apply(remove_special_char)


from konlpy.tag import Okt

okt = Okt()

train["title_token"] = train.title.apply(okt.morphs)
train["comment_token"] = train.comment.apply(okt.nouns)
    
train.head()
# train.to_excel("/USER/3_WEEK/MNC_NLP/train/train_1.xlsx")

,title,comment,bias,hate,label,title_token,comment_token
0,미스터 샤인션 변요한 김태리와 같은 양복 입고 학당 방문 이유는,김태리 정말 연기잘해 진짜,none,none,0,"[미스터, 샤인, 션, 변, 요한, 김태리, 와, 같은, 양복, 입고, 학당, 방문...","[김태리, 정말, 연기, 진짜]"
1,현장 극사실주의 현실 가장 보통의 연애 김래원 공효진 16년만의 랑데부 종합,공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,1,"[현장, 극, 사실, 주의, 현실, 가장, 보통, 의, 연애, 김래원, 공효진, 1...","[공효진, 발연기, 이질, 생각, 읍, 던데, 왜, 계속, 주연]"
2,손연재 리듬체조 학원 선생님 하고 싶은 일 해서 행복하다,누구처럼 돈만 밝히는 저급인생은 살아가지마시길 행복은 머니순이 아니니깐 작은거에 ...,others,hate,3,"[손연재, 리듬체조, 학원, 선생님, 하고, 싶은, 일, 해서, 행복하다]","[누구, 돈, 저급, 인생, 행복, 순이, 항상, 일만]"
3,섹션 김해숙 허스토리 촬영 후 우울증 얻었다,일본 축구 져라,none,none,0,"[섹션, 김해숙, 허, 스토리, 촬영, 후, 우울증, 얻었다]","[일본, 축구]"
4,단독 임현주 아나운서 노브라 챌린지 방송 덕에 낸 용기 자연스런 논의의 창...,난 절대로 임현주 욕하는인간이랑은 안논다,none,none,0,"[단독, 임현주, 아나운서, 노, 브라, 챌, 린지, 방송, 덕, 에, 낸, 용기,...","[난, 절대로, 임현주, 욕, 인간]"


In [73]:
train["content"] = train.title_token + train.comment_token
train.head()

,title,comment,bias,hate,label,title_token,comment_token,content
0,미스터 샤인션 변요한 김태리와 같은 양복 입고 학당 방문 이유는,김태리 정말 연기잘해 진짜,none,none,0,"[미스터, 샤인, 션, 변, 요한, 김태리, 와, 같은, 양복, 입고, 학당, 방문...","[김태리, 정말, 연기, 진짜]","[미스터, 샤인, 션, 변, 요한, 김태리, 와, 같은, 양복, 입고, 학당, 방문..."
1,현장 극사실주의 현실 가장 보통의 연애 김래원 공효진 16년만의 랑데부 종합,공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,1,"[현장, 극, 사실, 주의, 현실, 가장, 보통, 의, 연애, 김래원, 공효진, 1...","[공효진, 발연기, 이질, 생각, 읍, 던데, 왜, 계속, 주연]","[현장, 극, 사실, 주의, 현실, 가장, 보통, 의, 연애, 김래원, 공효진, 1..."
2,손연재 리듬체조 학원 선생님 하고 싶은 일 해서 행복하다,누구처럼 돈만 밝히는 저급인생은 살아가지마시길 행복은 머니순이 아니니깐 작은거에 ...,others,hate,3,"[손연재, 리듬체조, 학원, 선생님, 하고, 싶은, 일, 해서, 행복하다]","[누구, 돈, 저급, 인생, 행복, 순이, 항상, 일만]","[손연재, 리듬체조, 학원, 선생님, 하고, 싶은, 일, 해서, 행복하다, 누구, ..."
3,섹션 김해숙 허스토리 촬영 후 우울증 얻었다,일본 축구 져라,none,none,0,"[섹션, 김해숙, 허, 스토리, 촬영, 후, 우울증, 얻었다]","[일본, 축구]","[섹션, 김해숙, 허, 스토리, 촬영, 후, 우울증, 얻었다, 일본, 축구]"
4,단독 임현주 아나운서 노브라 챌린지 방송 덕에 낸 용기 자연스런 논의의 창...,난 절대로 임현주 욕하는인간이랑은 안논다,none,none,0,"[단독, 임현주, 아나운서, 노, 브라, 챌, 린지, 방송, 덕, 에, 낸, 용기,...","[난, 절대로, 임현주, 욕, 인간]","[단독, 임현주, 아나운서, 노, 브라, 챌, 린지, 방송, 덕, 에, 낸, 용기,..."


In [74]:
train_df = train[['content', 'label']]
train_df.head()

,content,label
0,"[미스터, 샤인, 션, 변, 요한, 김태리, 와, 같은, 양복, 입고, 학당, 방문...",0
1,"[현장, 극, 사실, 주의, 현실, 가장, 보통, 의, 연애, 김래원, 공효진, 1...",1
2,"[손연재, 리듬체조, 학원, 선생님, 하고, 싶은, 일, 해서, 행복하다, 누구, ...",3
3,"[섹션, 김해숙, 허, 스토리, 촬영, 후, 우울증, 얻었다, 일본, 축구]",0
4,"[단독, 임현주, 아나운서, 노, 브라, 챌, 린지, 방송, 덕, 에, 낸, 용기,...",0


In [75]:
train_df.iloc[4]["content"]

['단독',
 '임현주',
 '아나운서',
 '노',
 '브라',
 '챌',
 '린지',
 '방송',
 '덕',
 '에',
 '낸',
 '용기',
 '자연스런',
 '논의',
 '의',
 '창',
 '됐으면',
 '인터뷰',
 '난',
 '절대로',
 '임현주',
 '욕',
 '인간']

In [80]:
from gensim.models import Word2Vec
embedding_model = Word2Vec(train_df['content'],
                          sg=1,
                          # size=100,
                          window=2,
                          min_count=1,
                          workers=4)
print(embedding_model)

Word2Vec(vocab=12934, vector_size=100, alpha=0.025)


In [99]:
model_result = embedding_model.wv.most_similar("아빠")
for line in model_result:
    print(line)

print()
print()
model_result = embedding_model.wv.most_similar("엄마")
for line in model_result:
    print(line)

('된다', 0.962231457233429)
('좋은', 0.9603480696678162)
('울며', 0.9539052844047546)
('홍수현', 0.9533455967903137)
('문자', 0.9527677297592163)
('고충', 0.9520737528800964)
('쌍둥이', 0.9515073299407959)
('홍석천', 0.9513181447982788)
('감정', 0.9504828453063965)
('이소라', 0.950280487537384)


('랑', 0.9600263237953186)
('시집', 0.9510328769683838)
('아이', 0.9500100016593933)
('완전', 0.9480282664299011)
('자랑', 0.9444349408149719)
('먼저', 0.9444178938865662)
('엔딩', 0.9442457556724548)
('다나', 0.9440619349479675)
('많이', 0.9438155293464661)
('후배', 0.9435783624649048)


In [27]:
train = pd.read_excel("/USER/3_WEEK/MNC_NLP/train/train_1.xlsx", index_col=0)
train

,content,label
0,김태리 정말 연기잘해 진짜,0
1,공효진 발연기나이질생각이읍던데 왜계속주연일까,1
2,누구처럼 돈만 밝히는 저급인생은 살아가지마시길 행복은 머니순이 아니니깐 작은거에 ...,3
3,일본 축구 져라,0
4,난 절대로 임현주 욕하는인간이랑은 안논다,0
...,...,...
8362,아니 근데 튜닝한사람은 프러포즈받지도 결혼도못함 ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,3
8363,그러니깐 여자를 잘만나야되 징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯 근데...,5
8364,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다,0
8365,재미가 없어요,0


In [100]:
train_df.to_excel("/USER/3_WEEK/MNC_NLP/train/train_1_okt.xlsx")

In [102]:
CLS = bert_sequence_classification.Classification(model_name='kykim/bert-kor-base', min_sentence_length=5, MAX_LEN=128, batch_size=32, use_bert_tokenizer=True)
CLS.dataset(data_path='/USER/3_WEEK/MNC_NLP/train/train_1_okt.xlsx')
CLS.load_model(mode='train')
CLS.train(epochs=3, dataset_split=0.1)

6 labels, 8367 dataset
label counts:: Counter({0: 3422, 1: 2068, 3: 1437, 5: 1216, 2: 141, 4: 83})


Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

train-dataset is prepared

======== Epoch 1 / 3 ========
Training...


/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  Batch   100  of    236.    Elapsed: 0:01:08.
  Batch   200  of    236.    Elapsed: 0:02:18.

  Train loss: 1.30, Train Accuracy: 0.46
  Training epcoh took: 0:02:44

Running Validation...
  Validation Accuracy: 0.54
  Validation took: 0:00:06
              precision    recall  f1-score   support

           0       0.54      0.83      0.65       327
           1       0.38      0.21      0.27       225
           2       0.00      0.00      0.00        16
           3       0.44      0.26      0.33       145
           4       0.00      0.00      0.00         5
           5       0.67      0.67      0.67       119

    accuracy                           0.52       837
   macro avg       0.34      0.33      0.32       837
weighted avg       0.48      0.52      0.48       837


======== Epoch 2 / 3 ========
Training...


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  Batch   100  of    236.    Elapsed: 0:01:15.
  Batch   200  of    236.    Elapsed: 0:02:30.

  Train loss: 1.07, Train Accuracy: 0.58
  Training epcoh took: 0:02:57

Running Validation...
  Validation Accuracy: 0.57
  Validation took: 0:00:07
              precision    recall  f1-score   support

           0       0.60      0.69      0.64       327
           1       0.41      0.40      0.41       225
           2       0.00      0.00      0.00        16
           3       0.52      0.46      0.49       145
           4       0.00      0.00      0.00         5
           5       0.73      0.67      0.70       119

    accuracy                           0.55       837
   macro avg       0.38      0.37      0.37       837
weighted avg       0.54      0.55      0.54       837


======== Epoch 3 / 3 ========
Training...


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  Batch   100  of    236.    Elapsed: 0:01:16.
  Batch   200  of    236.    Elapsed: 0:02:31.

  Train loss: 0.92, Train Accuracy: 0.65
  Training epcoh took: 0:02:58

Running Validation...
  Validation Accuracy: 0.57
  Validation took: 0:00:07
              precision    recall  f1-score   support

           0       0.61      0.72      0.66       327
           1       0.41      0.36      0.38       225
           2       0.00      0.00      0.00        16
           3       0.50      0.43      0.46       145
           4       0.00      0.00      0.00         5
           5       0.66      0.71      0.69       119

    accuracy                           0.55       837
   macro avg       0.36      0.37      0.36       837
weighted avg       0.53      0.55      0.54       837


Training complete!


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [121]:
import pandas as pd
test = pd.read_csv("/USER/3_WEEK/MNC_NLP/test/test.csv")
test.title = test.title.apply(remove_special_char)
test.comment = test.comment.apply(remove_special_char)


from konlpy.tag import Okt

okt = Okt()

test["title_token"] = test.title.apply(okt.morphs)
test["comment_token"] = test.comment.apply(okt.nouns)
test["comment_token_1"] = test.comment.apply(okt.morphs)
    
test.head()

,ID,title,comment,title_token,comment_token,comment_token_1
0,0,류현경 박성훈 공개연애 4년차 애정전선 이상 의지 많이 된다 종합,둘다 넘 좋다 행복하세요,"[류현경, 박성훈, 공개, 연애, 4년, 차, 애정, 전선, 이상, 의지, 많이, ...",[],"[둘다, 넘, 좋다, 행복하세요]"
1,1,현금 유도 1인 1라면 골목식당 백종원 초심 잃은 도시락집에 경악 종합,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,"[현금, 유도, 1, 인, 1, 라면, 골목, 식당, 백종원, 초심, 잃은, 도시락...","[이하, 현금, 결제, 하라, 집, 우리나라]","[근데, 만, 원, 이하, 는, 현금, 결제, 만, 하라, 고, 써놓은, 집, 우리..."
2,2,입대 11 서은광의 슬픈 멜로디 비투비 눈물의 첫 체조경기장 콘서트 종합,누군데 얘네,"[입대, 11, 서은광, 의, 슬픈, 멜로디, 비투비, 눈물, 의, 첫, 체조, 경...",[얘],"[누군데, 얘, 네]"
3,3,아이콘택트 리쌍 길 3년 전 결혼설 부인한 이유 공개 결혼 출산 숨겼다,쑈 하지마라 짜식아 음주 1번은 실수 2번은 고의 3번은 인간쓰레기다 슬금슬금 ...,"[아이콘택트, 리쌍, 길, 3년, 전, 결혼설, 부인, 한, 이유, 공개, 결혼, ...","[쑈, 음주, 번은, 실수, 번은, 고의, 번은, 인간, 쓰레기, 슬금슬금, 생각,...","[쑈, 하지마라, 짜, 식아, 음주, 1, 번은, 실수, 2, 번은, 고의, 3, ..."
4,4,구하라 안검하수 반박 해프닝 당당하다 그렇게까지 설전 종합,안검하수 가지고 있는 분께 희망을 주고 싶은건가요 수술하면 이렇게 자연스러워진다고...,"[구하라, 안검, 하수, 반박, 해프닝, 당당하다, 그렇게, 까지, 설전, 종합]","[안검, 하수, 가지, 분, 희망, 가요, 수술, 눈, 동영상, 보시, 정작, 안검...","[안검, 하수, 가지, 고, 있는, 분, 께, 희망, 을, 주고, 싶은건, 가요, ..."


In [122]:
test["content"] = test.title_token + test.comment_token_1
test.head()

,ID,title,comment,title_token,comment_token,comment_token_1,content
0,0,류현경 박성훈 공개연애 4년차 애정전선 이상 의지 많이 된다 종합,둘다 넘 좋다 행복하세요,"[류현경, 박성훈, 공개, 연애, 4년, 차, 애정, 전선, 이상, 의지, 많이, ...",[],"[둘다, 넘, 좋다, 행복하세요]","[류현경, 박성훈, 공개, 연애, 4년, 차, 애정, 전선, 이상, 의지, 많이, ..."
1,1,현금 유도 1인 1라면 골목식당 백종원 초심 잃은 도시락집에 경악 종합,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,"[현금, 유도, 1, 인, 1, 라면, 골목, 식당, 백종원, 초심, 잃은, 도시락...","[이하, 현금, 결제, 하라, 집, 우리나라]","[근데, 만, 원, 이하, 는, 현금, 결제, 만, 하라, 고, 써놓은, 집, 우리...","[현금, 유도, 1, 인, 1, 라면, 골목, 식당, 백종원, 초심, 잃은, 도시락..."
2,2,입대 11 서은광의 슬픈 멜로디 비투비 눈물의 첫 체조경기장 콘서트 종합,누군데 얘네,"[입대, 11, 서은광, 의, 슬픈, 멜로디, 비투비, 눈물, 의, 첫, 체조, 경...",[얘],"[누군데, 얘, 네]","[입대, 11, 서은광, 의, 슬픈, 멜로디, 비투비, 눈물, 의, 첫, 체조, 경..."
3,3,아이콘택트 리쌍 길 3년 전 결혼설 부인한 이유 공개 결혼 출산 숨겼다,쑈 하지마라 짜식아 음주 1번은 실수 2번은 고의 3번은 인간쓰레기다 슬금슬금 ...,"[아이콘택트, 리쌍, 길, 3년, 전, 결혼설, 부인, 한, 이유, 공개, 결혼, ...","[쑈, 음주, 번은, 실수, 번은, 고의, 번은, 인간, 쓰레기, 슬금슬금, 생각,...","[쑈, 하지마라, 짜, 식아, 음주, 1, 번은, 실수, 2, 번은, 고의, 3, ...","[아이콘택트, 리쌍, 길, 3년, 전, 결혼설, 부인, 한, 이유, 공개, 결혼, ..."
4,4,구하라 안검하수 반박 해프닝 당당하다 그렇게까지 설전 종합,안검하수 가지고 있는 분께 희망을 주고 싶은건가요 수술하면 이렇게 자연스러워진다고...,"[구하라, 안검, 하수, 반박, 해프닝, 당당하다, 그렇게, 까지, 설전, 종합]","[안검, 하수, 가지, 분, 희망, 가요, 수술, 눈, 동영상, 보시, 정작, 안검...","[안검, 하수, 가지, 고, 있는, 분, 께, 희망, 을, 주고, 싶은건, 가요, ...","[구하라, 안검, 하수, 반박, 해프닝, 당당하다, 그렇게, 까지, 설전, 종합, ..."


In [132]:
test_df = test[['ID', 'content']]

def listToString(s): 
    
    str1 = "" 
    
    for ele in s: 
        str1 += ele + " "  
    
    return str1 
        
test_list = list()
for line in test["content"]:
    test_list.append(listToString(line))
# test_list

In [133]:
# 

In [134]:
# sentences = ['영화 재밌어요', '영화 재미없어요', '그냥 시간떼우기용', '완전 추천작']
sentences = test_list
saved_model_path='model/saved/3'

CLS = bert_sequence_classification.Classification(model_name='kykim/bert-kor-base', min_sentence_length=5, MAX_LEN=128, batch_size=64, use_bert_tokenizer=True)
CLS.load_model(mode='inference', saved_model_path=saved_model_path)
logit = CLS.inference(sentences=sentences)
print(logit)

[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 5, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 5, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 1, 0, 0, 5, 0, 0, 1, 1, 5, 0, 5, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 5, 1, 1, 0, 1, 0, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 5, 5, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 5, 0, 0, 0, 0, 5, 0, 0, 0, 5, 0, 1, 1, 0, 1, 0, 0, 0, 5, 0, 1, 0, 5, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 5, 3, 0, 1, 0, 0, 1, 5, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 5, 0, 0, 0, 0, 1, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 5, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 0, 0, 1, 0, 0, 5, 5, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 5, 0, 1, 0, 0, 3, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 

In [135]:
from collections import Counter
# a = Counter(logit)    # model.csv   5epochs
# b = Counter(logit_1)  # model_1.csv 15epochs
c = Counter(logit)
print(len(logit))

511


In [136]:
# print(a)
# print(b)
print(c)

Counter({0: 338, 1: 126, 5: 41, 3: 6})


In [137]:
submit = pd.read_csv("/USER/3_WEEK/MNC_NLP/test/sample_submission.csv")
submit

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,none
3,3,none,none
4,4,none,none
...,...,...,...
506,506,none,none
507,507,none,none
508,508,none,none
509,509,none,none


In [138]:
# 0 ['none' 'none']
# 1 ['none' 'hate']
# 2 ['others' 'none']
# 3 ['others' 'hate']
# 4 ['gender' 'none']
# 5 ['gender' 'hate']
count = 0
for i in range(len(logit)):
    if logit[i] == 0:
        count += 1
        submit.iloc[i] = [i, 'none', 'none']
    elif logit[i] == 1:
        submit.iloc[i] = [i, 'none', 'hate']
    elif logit[i] == 2:
        submit.iloc[i] = [i, 'others', 'none']
    elif logit[i] == 3:
        submit.iloc[i] = [i, 'others', 'hate']
    elif logit[i] == 4:
        submit.iloc[i] = [i, 'gender', 'none']
    elif logit[i] == 5:
        submit.iloc[i] = [i, 'gender', 'hate']
        
print(count)

338


In [139]:
submit

,ID,bias,hate
0,0,none,none
1,1,none,hate
2,2,none,hate
3,3,none,hate
4,4,none,hate
...,...,...,...
506,506,none,hate
507,507,none,none
508,508,none,hate
509,509,none,none


In [140]:
submit.to_csv("/USER/3_WEEK/MNC_NLP/submit/dlwpwlswkahfmrpTek.csv")

In [141]:
# from transformers import ElectraTokenizerFast, ElectraModel

# tokenizer_electra = ElectraTokenizerFast.from_pretrained("kykim/electra-kor-base")
# model = ElectraModel.from_pretrained("kykim/electra-kor-base")

In [142]:
# import torch
# import torch.nn as nn

# inputs = tokenizer_electra.encode("안녕하세요", return_tensors='pt')
# outputs = model(inputs)
# outputs.last_hidden_state.shape






In [144]:

# te = outputs.logits[0].tolist()
# for i in range(len(te)):
#     te[i] = round(te[i], 4)
# te